## Customized Chest CT Anomaly Segmentation Algorithm from AWS Marketplace 

This deep learning based solution provides chest CT analysis with precise segmentation of various anomalies and chest conditions.

This sample notebook shows you how to execute Customized Chest CT Anomaly Segmentation Algorithm from AWS Marketplace 

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**u
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: clean-sentiment-classification-labels. 

#### Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure dataset](#B.-Configure-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Execute optimization model](#3.-Execute-optimization-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Execute model](#3.2-Execute-model)
    1. [Visualize Output](#3.3-Inspect-the-Output-in-S3)
1. [Clean-up](#4.-Clean-up)
	1. [Unsubscribe to the listing (optional)](#Unsubscribe-to-the-listing-(optional))


#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page **Clean Sentiment Classification Labels**
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

## Setting up the Environment

In [ ]:
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()

# S3 prefixes
common_prefix = "medseglab_demo_test"
training_input_prefix = common_prefix + "/training-input-data"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"

### Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our Amazon SageMaker operations.

In [ ]:
sagemaker_session = sage.Session()

### Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using an open source dataset mentioned in github link: https://github.com/Mphasis-ML-Marketplace/medseglab-demo/tree/main

We can use use the tools provided by the Amazon SageMaker Python SDK to upload the data to a default bucket.

In [ ]:
TRAINING_WORKDIR = "data/training"

training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix)
print("Training Data Location " + training_input)

### Creating Training Job using Algorithm ARN

Please put in the algorithm arn you want to use below. This can either be an AWS Marketplace algorithm you subscribed to (or) one of the algorithms you created in your own account.

The algorithm arn listed below belongs to the Scikit Decision Trees product.

In [ ]:
algorithm_arn = "medseglab-v0"

Configure hyperparameters for training

In [ ]:
hyperparameters = {"bs": 16, "epochs": 2} # batch size and epochs

In [ ]:
from sagemaker.algorithm import AlgorithmEstimator

algo = AlgorithmEstimator(
    algorithm_arn=algorithm_arn,
    role=role,
    train_instance_count=1,
    train_instance_type="ml.g5.2xlarge",
    instance_count=1, 
    instance_type="ml.g5.2xlarge", 
    base_job_name="medseglab-v0-test",
    hyperparameters=hyperparameters
)

### Run Training Job

In [ ]:
print(
    "Now run the training job using algorithm arn %s in region %s"
    % (algorithm_arn, sagemaker_session.boto_region_name)
)
algo.fit({"training": training_input})

### Batch Transform Job
Now let's use the model built to run a batch inference job and verify it works.

In [ ]:
TRANSFORM_WORKDIR = "data/inference"

transform_input = sagemaker_session.upload_data(TRANSFORM_WORKDIR, key_prefix=batch_inference_input_prefix)
print("Batch Transform Data Location " + transform_input)

In [ ]:
transformer = algo.transformer(1, "ml.p2.xlarge")
transformer.transform(transform_input, content_type="application/zip")
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

### Inspect the Batch Transform Output in S3

In [ ]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = "{}/{}.out".format(parsed_url.path[1:], "frames.zip")

s3_client = sagemaker_session.boto_session.client("s3")

with open("response.zip", "wb") as FO:
    s3_client.download_fileobj(Bucket=sagemaker_session.default_bucket(), Key=file_key, Fileobj=FO)

In [ ]:
!mkdir batch_transform_output/
!unzip response.zip -d ./batch_transform_output/

### Live Inference Endpoint
Finally, we demonstrate the creation of an endpoint for live inference using this AWS Marketplace algorithm generated model

In [ ]:
from sagemaker.serializers import IdentitySerializer

predictor = algo.deploy(1, "ml.p2.xlarge", serializer=IdentitySerializer(content_type='image/png'))

In [ ]:
image_path = "img_payload.png"

with open(image_path, "rb") as f:
    image_payload_bytes = f.read()

In [ ]:
response = predictor.predict(image_payload_bytes)

In [ ]:
from PIL import Image
from io import BytesIO

In [ ]:
labeled_image = Image.open(BytesIO(response)).convert("RGB")
labeled_image

### Cleanup the endpoint

In [ ]:
algo.delete_endpoint()

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.